In [1]:
import sys
from pathlib import Path
import os


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config

if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: c:\Users\dtanu\Documents\RAJAN_TANUDIN-mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings

warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country = location['country']
city = location['city']
street = location['street']
aqicn_url = location['aqicn_url']
latitude = location['latitude']
longitude = location['longitude']

today = datetime.date.today()

location_str

2025-11-18 22:03:39,371 INFO: Initializing external client
2025-11-18 22:03:39,371 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-18 22:03:39,371 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 22:03:40,848 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1303708

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1303708


'{"country": "United States", "city": "San Francisco", "street": "1600 Divisadero Street", "aqicn_url": "https://api.waqi.info/feed/A399343", "latitude": "37.7627257", "longitude": "-122.458256818062"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,13.0,United States,San Francisco,1600 Divisadero Street,2025-11-18,https://api.waqi.info/feed/A399343


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🔄 Add Lagged Features from Historical Data</span>

To create lagged features for today's prediction, we need to fetch the PM2.5 values from the previous 3 days from the feature store.

In [7]:
# Fetch the air quality feature group
air_quality_fg = fs.get_feature_group(name="air_quality", version=1)

# Calculate dates for the last 3 days
yesterday = today - datetime.timedelta(days=1)
two_days_ago = today - datetime.timedelta(days=2)
three_days_ago = today - datetime.timedelta(days=3)

# Fetch historical PM2.5 values from the feature store
query = air_quality_fg.select(['date', 'pm25']).filter(
    (air_quality_fg.country == country) & 
    (air_quality_fg.city == city) & 
    (air_quality_fg.street == street) &
    (air_quality_fg.date >= three_days_ago) &
    (air_quality_fg.date < today)
)

historical_aq_df = query.read()
historical_aq_df = historical_aq_df.sort_values('date')

print(f"Historical data retrieved:")
print(historical_aq_df)

# Extract lagged values
if len(historical_aq_df) >= 3:
    pm25_lag1 = historical_aq_df[historical_aq_df['date'] == pd.Timestamp(yesterday).normalize()]['pm25'].values
    pm25_lag2 = historical_aq_df[historical_aq_df['date'] == pd.Timestamp(two_days_ago).normalize()]['pm25'].values
    pm25_lag3 = historical_aq_df[historical_aq_df['date'] == pd.Timestamp(three_days_ago).normalize()]['pm25'].values
    
    # Add lagged features to today's dataframe
    aq_today_df['pm25_lag1'] = pm25_lag1[0] if len(pm25_lag1) > 0 else None
    aq_today_df['pm25_lag2'] = pm25_lag2[0] if len(pm25_lag2) > 0 else None
    aq_today_df['pm25_lag3'] = pm25_lag3[0] if len(pm25_lag3) > 0 else None
    
    print(f"\nLagged features added:")
    print(f"PM2.5 lag1 (yesterday): {aq_today_df['pm25_lag1'].values[0]}")
    print(f"PM2.5 lag2 (2 days ago): {aq_today_df['pm25_lag2'].values[0]}")
    print(f"PM2.5 lag3 (3 days ago): {aq_today_df['pm25_lag3'].values[0]}")
else:
    print(f"Warning: Not enough historical data to create lagged features. Found {len(historical_aq_df)} days.")
    aq_today_df['pm25_lag1'] = None
    aq_today_df['pm25_lag2'] = None
    aq_today_df['pm25_lag3'] = None

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
Historical data retrieved:
                       date  pm25
0 2025-11-16 00:00:00+00:00  2.94
2 2025-11-17 00:00:00+00:00  2.52
1 2025-11-18 00:00:00+00:00  3.49

Lagged features added:
PM2.5 lag1 (yesterday): None
PM2.5 lag2 (2 days ago): None
PM2.5 lag3 (3 days ago): None
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 
Historical data retrieved:
                       date  pm25
0 2025-11-16 00:00:00+00:00  2.94
2 2025-11-17 00:00:00+00:00  2.52
1 2025-11-18 00:00:00+00:00  3.49

Lagged features added:
PM2.5 lag1 (yesterday): None
PM2.5 lag2 (2 days ago): None
PM2.5 lag3 (3 days ago): None


,pm25,country,city,street,date,url,pm25_lag1,pm25_lag2,pm25_lag3
0,13.0,United States,San Francisco,1600 Divisadero Street,2025-11-18,https://api.waqi.info/feed/A399343,None,None,None


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [8]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 37.5°N -122.25°E
Elevation 129.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,8.50,0.0,4.104631,127.875046,San Francisco
1,2025-11-19,9.40,0.0,5.052841,175.914459,San Francisco
2,2025-11-20,10.80,0.8,4.104631,164.744827,San Francisco
3,2025-11-21,9.05,0.0,0.509117,224.999893,San Francisco
4,2025-11-22,9.60,0.0,0.360000,90.000000,San Francisco
5,2025-11-23,8.40,0.0,0.720000,180.000000,San Francisco
6,2025-11-24,10.05,0.0,1.138420,288.435028,San Francisco


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 22:03:47,740 INFO: 	4 expectation(s) included in expectation_suite.
Validation failed.Validation failed.

Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303708/fs/1291338/fg/1719062
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303708/fs/1291338/fg/1719062


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00



Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303708/jobs/named/air_quality_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303708/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_lag2",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 759120
         }
       },
       "result": {
         "observed_value": null,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T09:03:47.000740Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_m

In [11]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 22:04:01,071 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303708/fs/1291338/fg/1719040
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1303708/fs/1291338/fg/1719040


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303708/jobs/named/weather_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1303708/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 22:04:16,875 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 22:04:16,875 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 22:04:20,084 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 22:04:20,084 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 22:05:48,940 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 22:05:48,940 INFO: Waiting for execution to fin

(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 759109
         }
       },
       "result": {
         "observed_value": 0.35999998450279236,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T09:04:01.000071Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type"

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
